In [2]:
import cv2
import numpy as np
import dlib
# import datetime as dt
from datetime import datetime as dt
now_dt = dt.now().strftime('%m/%d/%Y %H:%M:%S')

cap = cv2.VideoCapture('../video/videoplayback (1)_Trim.mp4')   # play video file
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')

font = 2;    lt = 16
print(f'frame_w\t\t: {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}\n'
      f'frame_fps\t: {cap.get(cv2.CAP_PROP_FPS)}\n'
      f'frame_count\t: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}')


# 指定 ROI 座標位置
RECT = ((200, 20), (570, 390))
(left, top), (right, bottom) = RECT

def roiarea(frame):                  # 取出 ROI 子畫面
    return frame[top:bottom, left:right]

def replaceroi(frame, roi):             # 將 ROI 區域貼回到原畫面
    frame[top:bottom, left:right] = roi
    return frame

bg = None

ratio = cap.get(cv2.CAP_PROP_FRAME_WIDTH) / cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # cap.get得到相機/視訊檔案的屬性
w = 800;  h = int(w / ratio)

out = cv2.VideoWriter('../video/test4.mp4', fourcc, fps, (w, h))

detector = dlib.get_frontal_face_detector()
font = 2;    lt = 16
while True:
    ret, frame = cap.read()                          #  read frame : ret: True/ False,  frame:image
    if not ret or cv2.waitKey(30) == 27: break             # wait for 1 ms     
    
    frame = cv2.flip(frame, 1)                       # 0 : 上下左右顛倒,  -1 : 上下顛倒   
    frame = cv2.resize(frame, (w, h))    
    cv2.putText(frame, f'{cap.get(cv2.CAP_PROP_POS_FRAMES):.0f} frames, {cap.get(cv2.CAP_PROP_POS_MSEC):.0f} ms', 
                (50, 250), font, .8, (0,0,255), 2, lt)
 
    fms = cap.get(cv2.CAP_PROP_POS_MSEC)
    
    if fms <3000 and fms > 500:
        roi = roiarea(frame)                   # 取出子畫面
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)        # BGR to HSV

        frame = replaceroi(frame, roi)         # 將處理完的子畫面貼回到原本畫面中

        cv2.rectangle(frame, RECT[0], RECT[1], (0,0,255), 2)      # 在 ROI 範圍處畫個框
        cv2.imshow('frame', frame)
        
        out.write(frame)    
    elif fms > 3000 and fms < 7000:
        face_rects, scores, idx = detector.run(frame, 0)  # 偵測人臉

        for i, d in enumerate(face_rects):                  # 取出所有偵測的結果
            x1, y1, x2, y2 = d.left(), d.top(), d.right(), d.bottom()

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA) # 以方框標示偵測的人臉
            cv2.putText(frame, f'{scores[i]:.2f}, ({idx[i]:0.0f})', (x1, y1), font,    # 標示分數
                0.7, (255, 255, 255), 1, lt)

        cv2.imshow('frame', frame)
        out.write(frame)        
    elif fms > 7000 and fms < 11000:
        roi = roiarea(frame)                   # 取出子畫面
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2XYZ)          # BGR to XYZ

        frame = replaceroi(frame, roi)         # 將處理完的子畫面貼回到原本畫面中

        cv2.rectangle(frame, RECT[0], RECT[1], (0,0,255), 2)      # 在 ROI 範圍處畫個框
        cv2.imshow('frame', frame)
        out.write(frame)        
    elif fms > 11000 and fms < 14000:
        frame = cv2.flip(frame, 0) 
        cv2.imshow('frame', frame)
        out.write(frame)
    elif fms > 14000 and fms < 20000:
        frame = cv2.Canny(frame, 100, 200)
        cv2.imshow('frame', frame)
        out.write(frame)    
    elif fms > 20000 and fms < 25000:
        sobelX = cv2.Sobel(frame, cv2.CV_64F, 1, 0)
        sobelY = cv2.Sobel(frame, cv2.CV_64F, 0, 1)
        sobelX = cv2.convertScaleAbs(sobelX)
        sobelY = cv2.convertScaleAbs(sobelY)
        sobelXY = cv2.addWeighted(sobelX, 0.5, sobelY, 0.5, 0)
        frame = sobelXY
        cv2.imshow('frame', frame)
        out.write(frame)        
    elif fms > 25000 and fms < 30000:
        lap = cv2.Laplacian(frame, cv2.CV_64F)
        lap = cv2.convertScaleAbs(lap)
        frame = lap
        cv2.imshow('frame', frame)
        out.write(frame)        
    elif fms > 30000 and fms < 35000:
        d=5
        img_D3 = cv2.GaussianBlur(frame, (d, d), 0.3)
        img_D5 = cv2.GaussianBlur(frame, (d, d), 0.5)    
        img_D5_3 = img_D5 - img_D3   # try img_G0 + img_G1
        frame = img_D5_3
        cv2.imshow('frame', frame)
        out.write(frame)
    elif fms > 35000 and fms < 40000:


        d=5
        img_D7 = cv2.GaussianBlur(frame, (d, d), 0.7)   
        img_D9 = cv2.GaussianBlur(frame, (d, d), 0.9)     
        
        img_D9_7 = img_D9 - img_D7   # try img_G0 + img_G1
        frame = img_D9_7
        cv2.imshow('frame', frame)
        out.write(frame)        
    elif fms > 40000 and fms < 45000:
        i = (fms-40000)/5
        k = (fms-40000)/5
        pts1 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
        pts2 = np.float32([[0+i, 0+10], [w-i, k] ,[k, h-i], [w-k, h-k]])

        M = cv2.getPerspectiveTransform(pts1, pts2)
        affine_img = cv2.warpPerspective(frame, M, (w+100, h+100))    # 3 * 3 Matrix
        frame = affine_img

        cv2.imshow('frame', frame)
        out.write(frame)        
       

    elif fms > 45000 and fms < 50000:
        newHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        newHSV[:,:,2] = 255          # value 亮度 最亮

        art = cv2.cvtColor(newHSV, cv2.COLOR_HSV2BGR)
        frame = art
        cv2.imshow('frame', frame)
        out.write(frame)
    elif fms > 50000 and fms < 53000:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        th, thresh = cv2.threshold(gray, 225, 255, cv2.THRESH_BINARY_INV)
        mask = cv2.erode(thresh.copy(), None, iterations=1)    # None : kernel  default is a simple 3x3 matrix
        frame = mask
        cv2.imshow('frame', frame)
        out.write(frame)
#     elif fms > 53000 and fms < 53500:
#         blurred = np.hstack([
#             cv2.GaussianBlur(frame, (5, 5), 0)
#         ])

#         cv2.imshow('frame', blurred)
        
    elif fms > 54000 and fms < 56000:
   
        blurred = np.hstack([
            cv2.GaussianBlur(frame, (9, 9), 0)
        ])
        frame = blurred
        cv2.imshow('frame', frame)
        out.write(frame)
         
        
        
    elif fms > 56000 and fms < 60000:  #rotation
        
        i = (fms-56000) 
        k = (fms-56000)/5000
        M1 = cv2.getRotationMatrix2D((w/2, h/2), i, k) #表示旋轉的中心點,表示旋轉的角度,圖像縮放因子
        rotate_img1 = cv2.warpAffine(frame, M1, (w, h))
        frame = rotate_img1
        cv2.imshow('frame', frame) 
        out.write(frame)

        
    else:
        cv2.imshow('frame', frame)
        out.write(frame)
        
     

        
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)   

frame_w		: 1920.0
frame_fps	: 30.0
frame_count	: 1817.0


-1